In [1]:
import requests
import pandas as pd
from textblob import TextBlob

In [2]:

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

In [3]:
response = requests.get('https://api.twitter.com/1.1/search/tweets.json?q=""%23Corona"&lang=en&result_type=recent&count=100', auth=BearerAuth('INSERT BEARER AUTH HERE'))

#Verify if response is 200
print(response)

<Response [200]>


In [4]:
# df_teste = pd.json_normalize(dict_tweet)
# df_teste
import json
dict_tweet = response.json()
df_teste = pd.DataFrame(dict_tweet['statuses']).to_numpy()
df_teste[1]

array(['Mon Jan 25 04:44:47 +0000 2021', 1353564451803422722,
       '1353564451803422722',
       'somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #FunniestVideos #Video #VIDEOS #TrendingNow #Exclusive #coronavirus… https://t.co/oRqrVIvOrz',
       True,
       {'hashtags': [{'text': 'fun', 'indices': [30, 34]}, {'text': 'funny', 'indices': [35, 41]}, {'text': 'FunniestVideos', 'indices': [42, 57]}, {'text': 'Video', 'indices': [58, 64]}, {'text': 'VIDEOS', 'indices': [65, 72]}, {'text': 'TrendingNow', 'indices': [73, 85]}, {'text': 'Exclusive', 'indices': [86, 96]}, {'text': 'coronavirus', 'indices': [97, 109]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/oRqrVIvOrz', 'expanded_url': 'https://twitter.com/i/web/status/1353564451803422722', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [111, 134]}]},
       {'iso_language_code': 'en', 'result_type': 'recent'},
       '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
       nan, No

In [5]:
i = 1
twitter_texts = list()
created_time = list()
for lista in dict_tweet['statuses']:
    print(i, '-', lista['text'])
    i += 1
    twitter_texts.append(lista['text'])
    created_time.append(lista['created_at'])
#dict_statuses = dict_tweet['statuses']


1 - #Vaccination drives are in full swing across the #country. Front line #workers are the first segment to get… https://t.co/8UogiZlmA7
2 - somebody stops the boat 🤣🤣🤣🤣

#fun #funny #FunniestVideos #Video #VIDEOS #TrendingNow #Exclusive #coronavirus… https://t.co/oRqrVIvOrz
3 - @thewire_in So wire finally found somebody speaking their agenda... Whatever be it, close to 20L Indians have taken… https://t.co/P6xqHFt22J
4 - RT @CovidIndiaSeva: Kindly share your itinerary with the following #COVID19 Control Rooms for assistance:
▪️ #Chennai: 1077, 044-25243454…
5 - Kindly share your itinerary with the following #COVID19 Control Rooms for assistance:
▪️ #Chennai: 1077, 044-252434… https://t.co/s67XHZlIu3
6 - RT @constanzag77: Over 100 Scientists, Doctors, &amp; Leading Authorities Call For Increased Vitamin D Use To Combat COVID-19 https://t.co/L5Am…
7 - #Corona free bharat #Corona  please look at this @CMOMaharashtra @MumbaiPolice @mybmc @narendramodi https://t.co/vsvg5ZZYPh
8 - ♦️♦️ 2021-

In [6]:
# get the statuses and assigns to df

import numpy as np

d = {'texts': twitter_texts}
df = pd.DataFrame(data=d)
print(df.head())
df

                                               texts
0  #Vaccination drives are in full swing across t...
1  somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...
2  @thewire_in So wire finally found somebody spe...
3  RT @CovidIndiaSeva: Kindly share your itinerar...
4  Kindly share your itinerary with the following...


,texts
0,#Vaccination drives are in full swing across t...
1,somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...
2,@thewire_in So wire finally found somebody spe...
3,RT @CovidIndiaSeva: Kindly share your itinerar...
4,Kindly share your itinerary with the following...
...,...
66,Corona adds up to much more than a deadly viru...
67,RT @ChiragMRA: @mybmcHealthDept\n@mybmcWardRS\...
68,@mybmcHealthDept\n@mybmcWardRS\n12-13 Days sin...
69,RT @DrTamilisaiGuv: #vaccine hesitancy should ...


In [7]:
# Cleaning the tweets
# Remove URLs, numbers, user_ids, punctuations
import re

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:")
    
#     # Remove break lines \n
#     tweets = np.vectorize(remove_pattern)(tweets, "\n")
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    #tweets = np.core.defchararray.replace(tweets, r"[^\w\s]", " ")
    #tweets = np.vectorize(remove_pattern)(tweets, "")
    
    return tweets

df['texts'] = clean_tweets(df['texts'])
print(df['texts'])

0     #Vaccination drives are in full swing across t...
1     somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...
2      So wire finally found somebody speaking their...
3      Kindly share your itinerary with the followin...
4     Kindly share your itinerary with the following...
                            ...                        
66    Corona adds up to much more than a deadly viru...
67     \n\n12-13 Days since issue raised lot of peop...
68    \n\n12-13 Days since issue raised lot of peopl...
69     #vaccine hesitancy should be clarified by Drs...
70     I’m raising my kid to hate #COVID, #maskmanda...
Name: texts, Length: 71, dtype: object


In [8]:
# Sentimental analysis using Vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
#     print("Overall sentiment dictionary is : ", sentiment_dict) 
#     print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
#     print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
#     print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
  
#     print("Sentence Overall Rated As", end = " ") 
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        return "positive"
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        return "negative"
  
    else : 
        return "neutral" 


# Scores for the tweets

df['classification'] = df['texts'].apply(sentiment_scores)
#df.drop(['Sentiments', 'C'], axis=1)
df

,texts,classification
0,#Vaccination drives are in full swing across t...,neutral
1,somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...,positive
2,So wire finally found somebody speaking their...,neutral
3,Kindly share your itinerary with the followin...,positive
4,Kindly share your itinerary with the following...,positive
...,...,...
66,Corona adds up to much more than a deadly viru...,neutral
67,\n\n12-13 Days since issue raised lot of peop...,neutral
68,\n\n12-13 Days since issue raised lot of peopl...,neutral
69,#vaccine hesitancy should be clarified by Drs...,positive


In [9]:
# Extra columns

d = {'created_at':created_time}
df['created_at'] = pd.DataFrame(data=d)
print(df.head())

df

                                               texts classification  \
0  #Vaccination drives are in full swing across t...        neutral   
1  somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...       positive   
2   So wire finally found somebody speaking their...        neutral   
3   Kindly share your itinerary with the followin...       positive   
4  Kindly share your itinerary with the following...       positive   

                       created_at  
0  Mon Jan 25 04:49:30 +0000 2021  
1  Mon Jan 25 04:44:47 +0000 2021  
2  Mon Jan 25 04:44:14 +0000 2021  
3  Mon Jan 25 04:44:09 +0000 2021  
4  Mon Jan 25 04:43:41 +0000 2021  


,texts,classification,created_at
0,#Vaccination drives are in full swing across t...,neutral,Mon Jan 25 04:49:30 +0000 2021
1,somebody stops the boat 🤣🤣🤣🤣\n\n#fun #funny #F...,positive,Mon Jan 25 04:44:47 +0000 2021
2,So wire finally found somebody speaking their...,neutral,Mon Jan 25 04:44:14 +0000 2021
3,Kindly share your itinerary with the followin...,positive,Mon Jan 25 04:44:09 +0000 2021
4,Kindly share your itinerary with the following...,positive,Mon Jan 25 04:43:41 +0000 2021
...,...,...,...
66,Corona adds up to much more than a deadly viru...,neutral,Mon Jan 25 02:45:52 +0000 2021
67,\n\n12-13 Days since issue raised lot of peop...,neutral,Mon Jan 25 02:42:37 +0000 2021
68,\n\n12-13 Days since issue raised lot of peopl...,neutral,Mon Jan 25 02:40:20 +0000 2021
69,#vaccine hesitancy should be clarified by Drs...,positive,Mon Jan 25 02:36:45 +0000 2021


In [10]:
# Sentimental Analysis with TextBlob
# I also need to clean the data
import pandas as pd
from textblob import TextBlob

def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment
    except:
        return None

    
#df['sentiment'] = df['texts'].apply(sentiment_calc)
#print(df)

In [11]:

df.to_excel("twitterSentimentals.xlsx", index=False)